In [ ]:
from src.models.math_roberta import MATH_ROBERTA_CONFIG
from src.models.numt5 import NUM_T5_CONFIG
! pip3 install evaluate
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir(next((root for root, _, files in os.walk(".") if "dsait4090_project_location" in files), "."))
    print(f'Google Colab: {os.getcwd()}')
except ImportError:
    print(f'Local: {os.getcwd()}')

In [ ]:
%load_ext autoreload
%autoreload

import logging
import torch
from torch import nn
from time import time

from src.models.pasta import PASTA_CONFIG
from src.models.deberta_v3 import DEBERTA_V3_CONFIG
from src.models.flant5 import FLAN_T5_CONFIG
from src.models.elastic_roberta import ELASTIC_ROBERTA_CONFIG
from src.models.bart_large_mnli import BART_LARGE_MNLI_CONFIG
from src.models.gpt2 import GPT2_CONFIG
from src.models.roberta_large_mnli import ROBERTA_LARGE_MNLI_CONFIG
from src.common import DATA_PATH, read_data, get_device, DOC, NO_DECOMPOSITION, GPT3_5_TURBO_DECOMP, FLANT5_DECOMP, CUSTOM_DECOMP
from src.classification_training import ClassificationTraining
from src.quantemp_processor import QuantempProcessor

logging.basicConfig(level=logging.ERROR)

device = get_device()

def data_path(decomposition: str, split: str):
    if decomposition == 'doc':
        path = f'raw_data/{split}_claims.json'
    elif decomposition == 'no_decomposition':
        return f'no_decomposition/{split}_evidences_no_decomposition.json'
    else:
        path = f'{decomposition}/{split}_evidences_decomposed_{decomposition}.json'

    return os.path.join(DATA_PATH, path)

In [ ]:
%autoreload

config = [
    (ROBERTA_LARGE_MNLI_CONFIG, DOC),
    (ROBERTA_LARGE_MNLI_CONFIG, NO_DECOMPOSITION),
    (ROBERTA_LARGE_MNLI_CONFIG, FLANT5_DECOMP),
    (ROBERTA_LARGE_MNLI_CONFIG, GPT3_5_TURBO_DECOMP),
    (ROBERTA_LARGE_MNLI_CONFIG, CUSTOM_DECOMP),

    (FLAN_T5_CONFIG, DOC),
    (FLAN_T5_CONFIG, NO_DECOMPOSITION),
    (FLAN_T5_CONFIG, FLANT5_DECOMP),
    (FLAN_T5_CONFIG, GPT3_5_TURBO_DECOMP),
    (FLAN_T5_CONFIG, CUSTOM_DECOMP),

    (DEBERTA_V3_CONFIG, DOC),
    (DEBERTA_V3_CONFIG, NO_DECOMPOSITION),
    (DEBERTA_V3_CONFIG, FLANT5_DECOMP),
    (DEBERTA_V3_CONFIG, GPT3_5_TURBO_DECOMP),
    (DEBERTA_V3_CONFIG, CUSTOM_DECOMP),

    (GPT2_CONFIG, DOC),

    (BART_LARGE_MNLI_CONFIG, DOC),
    (BART_LARGE_MNLI_CONFIG, NO_DECOMPOSITION),
    (BART_LARGE_MNLI_CONFIG, FLANT5_DECOMP),
    (BART_LARGE_MNLI_CONFIG, GPT3_5_TURBO_DECOMP),
    (BART_LARGE_MNLI_CONFIG, CUSTOM_DECOMP),

    (PASTA_CONFIG, DOC),
    (PASTA_CONFIG, NO_DECOMPOSITION),
    (PASTA_CONFIG, FLANT5_DECOMP),
    (PASTA_CONFIG, GPT3_5_TURBO_DECOMP),
    (PASTA_CONFIG, CUSTOM_DECOMP),

    (MATH_ROBERTA_CONFIG, DOC),
    (MATH_ROBERTA_CONFIG, NO_DECOMPOSITION),
    (MATH_ROBERTA_CONFIG, FLANT5_DECOMP),
    (MATH_ROBERTA_CONFIG, GPT3_5_TURBO_DECOMP),
    (MATH_ROBERTA_CONFIG, CUSTOM_DECOMP),

    (NUM_T5_CONFIG, DOC),
    (NUM_T5_CONFIG, NO_DECOMPOSITION),
    (NUM_T5_CONFIG, FLANT5_DECOMP),
    (NUM_T5_CONFIG, GPT3_5_TURBO_DECOMP),
    (NUM_T5_CONFIG, CUSTOM_DECOMP),

    (ELASTIC_ROBERTA_CONFIG, DOC),
    (ELASTIC_ROBERTA_CONFIG, NO_DECOMPOSITION),
    (ELASTIC_ROBERTA_CONFIG, FLANT5_DECOMP),
    (ELASTIC_ROBERTA_CONFIG, GPT3_5_TURBO_DECOMP),
    (ELASTIC_ROBERTA_CONFIG, CUSTOM_DECOMP),
]

print(len(config))
for model, decomposition in config:
    print(f'{model["name"]} - {decomposition}')

In [ ]:
%autoreload

DATA_LIMIT = None

for i, (model_config, decomposition) in enumerate(config):
    t0 = time()
    model_name = model_config['name']
    model_type = model_config['model']
    tokenizer_type = model_config['tokenizer']

    print(f"{'='*60}\n({i+1} / {len(config)})  {model_name.upper()} | {decomposition.upper()}   \n{'='*60}")

    train_claims = read_data(data_path(decomposition, 'train'))[:DATA_LIMIT]
    val_claims = read_data(data_path(decomposition, 'val'))[:DATA_LIMIT]
    test_claims = read_data(data_path(decomposition, 'test'))[:DATA_LIMIT]

    data_processor = QuantempProcessor(tokenizer_type(), decomposition=decomposition)
    train_dataset = data_processor.transform(train_claims)
    val_dataset = data_processor.transform(val_claims)
    test_dataset = data_processor.transform(test_claims)

    model = model_type().to(device)

    training = ClassificationTraining(
        model_name=f"{model_name}_{decomposition}",
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        model=model,
        optimizer=torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8),
        loss_function=nn.CrossEntropyLoss(),
        batch_size=8,
        device=device,
        random_state=256
    )

    # training.start_new_training()
    # training.train(epochs=20, patience=2)
    training.evaluate_best_model(test_claims, test_dataset)

    t1 = time()
    print(f"Time: {(t1 - t0) / 60:.2f} min")

